# Twitter Sentiment Analysis
En este notebook vas a ver un ejemplo de los procesos necesarios para realizar un análisis de sentimientos sobre Tweets. Para ello tendremos que seguir los siguientes pasos:
1. Conseguir un Corpus: no es más que una base de datos de texto etiquetado
2. Limpiar los datos
3. Entrenar un modelo con el corpus
4. Atacar a la API de Twitter
5. Predecir los nuevos Tweets

**Estos programas son muy útiles en campañas de marketing, para monitorizar el lanzamiento de un nuevo producto, realizar seguimiento en Twitter de eventos, o simplemente tener monitorizadas ciertas cuentas o hashtags para tener un programa de análisis real time.**

## 1. Corpus
Para conseguir el corpus tendremos que registrarnos en la [página del TASS](http://tass.sepln.org/tass_data/download.php), que se trata de una asociación de análisis semántico que encargada de recopilar texto y mantenerlo etiquetado. 

Para datasets en ingles lo tenemos más fácil ya que con librerías como [TextBlob](https://textblob.readthedocs.io/en/dev/) podemos predecir directamente la polaridad del Tweet, con modelos ya preentrenados. En el caso del castellano necesitamos acudir a un corpus etiquetado para entrenar nuestro modelo.

Registrate en el TASS y accede a sus corpus a través de un link que te llegará al correo tras el registro.

![imagen](img/tass_register.png)


Una vez estes registrado, descárgate el corpus de tweets en español de entrenamiento. En este punto lo ideal es coger un corpus que se adapte lo máximo posible a los tipos de tweets que intentamos predecir, es decir, si queremos predecir tweets sobre política, procurar elegir un corpus que tenga vocabulario de política.

En este notebook se va a elegir un corpus genérico con no demasiados registros para aligerar la limpieza y entrenamiento de los modelos.

![imagen](img/download_train_spanish.png)

### Importamos librerias

In [ ]:
# Importamos las librerías necesarias para el proyecto

# pandas: para manipulación y análisis de datos en formato tabular (DataFrames)
import pandas as pd

# xml.etree.ElementTree: para leer y parsear archivos XML (nuestro corpus viene en este formato)
import xml.etree.ElementTree as ET

# seaborn: para crear visualizaciones estadísticas atractivas
import seaborn as sns

# sklearn.svm: Support Vector Machines para clasificación
from sklearn.svm import LinearSVC  # Versión lineal (más rápida)
from sklearn.svm import SVC  # Versión con kernel (más flexible)

# Pipeline: para encadenar múltiples pasos del proceso de ML en un solo objeto
from sklearn.pipeline import Pipeline

# GridSearchCV: para búsqueda exhaustiva de los mejores hiperparámetros mediante validación cruzada
from sklearn.model_selection import GridSearchCV

### Leemos el dataset

In [ ]:
# Parseamos el archivo XML que contiene el corpus de tweets etiquetados
# El corpus viene descargado del TASS (Workshop on Semantic Analysis at SEPLN)

# parse(): lee el archivo XML y lo convierte en un árbol de elementos
tree = ET.parse('data/general-train-tagged.xml')

# getroot(): obtiene el elemento raíz del árbol XML (el nodo principal)
root = tree.getroot()

In [ ]:
# Creamos un diccionario vacío para almacenar los datos extraídos del XML
# Cada clave será una columna del DataFrame final
raw_dict = {
    'User': [],        # Usuario que escribió el tweet
    'Content': [],     # Contenido del tweet (el texto)
    'Date': [],        # Fecha de publicación
    'Lang': [],        # Idioma del tweet
    'Polarity': [],    # Polaridad del sentimiento (P+, P, NEU, N, N+, NONE)
    'Type': []         # Tipo de tweet (AGREEMENT, DISAGREEMENT, etc.)
}

# Iteramos sobre cada elemento 'tweet' en el XML
for i in root.iter('tweet'):
    # Extraemos cada campo del tweet mediante find() y obtenemos su texto con .text
    user = i.find('user').text
    content = i.find('content').text
    date = i.find('date').text
    lang = i.find('lang').text
    
    # La polaridad está anidada dentro de sentiments > polarity > value
    polarity = i.find('sentiments').find('polarity').find('value').text
    
    # El tipo también está dentro de sentiments > polarity > type
    tweet_type = i.find('sentiments').find('polarity').find('type').text
    
    # Añadimos cada valor a su lista correspondiente en el diccionario
    raw_dict['User'].append(user)
    raw_dict['Content'].append(content)
    raw_dict['Date'].append(date)
    raw_dict['Lang'].append(lang)
    raw_dict['Polarity'].append(polarity)
    raw_dict['Type'].append(tweet_type)

# Convertimos el diccionario en un DataFrame de pandas para facilitar su manipulación
df = pd.DataFrame(raw_dict)

# Mostramos la forma del DataFrame (número de filas y columnas)
print(df.shape)

# Visualizamos las primeras 5 filas para entender la estructura de los datos
df.head()

In [ ]:
# Configuramos pandas para mostrar el contenido completo de las columnas
# Por defecto, pandas trunca el texto largo con '...'
# max_colwidth=None permite ver el texto completo de los tweets
pd.set_option('max_colwidth', None)

In [ ]:
# Visualizamos de nuevo las primeras filas, ahora con el texto completo
# Esto nos ayuda a entender mejor el contenido real de los tweets
df.head()

### Columna de polaridad

In [ ]:
# Exploramos los valores únicos de la columna Polarity
# Esto nos muestra qué categorías de sentimiento tenemos en el dataset
# Resultado esperado: NONE, NEU, P, P+, N, N+
# P+ = Muy positivo, P = Positivo, NEU = Neutral, N = Negativo, N+ = Muy negativo, NONE = Sin sentimiento
df.Polarity.unique()

In [ ]:
# Creamos un gráfico de barras para visualizar la distribución de polaridades
# countplot: cuenta cuántos tweets hay de cada categoría y los representa en barras
# x='Polarity': usamos la columna Polarity en el eje X
# Esto nos ayuda a ver si hay desbalanceo de clases en nuestro dataset
sns.countplot(x = 'Polarity', data=df);

#### Columna de tipo

In [ ]:
# Visualizamos la distribución de la columna Type
# Esta columna indica si el tweet expresa acuerdo (AGREEMENT) o desacuerdo (DISAGREEMENT)
# Es útil para entender el contexto de los sentimientos
sns.countplot(x = 'Type', data=df);

## 2. Limpieza de datos
#### Polaridad
Vamos a clasificar los Tweets como buenos o malos, por lo que haremos la siguiente agrupación de la polaridad

In [ ]:
# Función para simplificar la polaridad a un problema binario (positivo vs negativo)
# En lugar de 6 categorías, vamos a tener solo 2: 0 (positivo) y 1 (negativo)

def polaridad_fun(x):
    # Si la polaridad es P (positivo) o P+ (muy positivo), devolvemos 0
    if x in ('P', 'P+'):
        return 0
    # Si la polaridad es N (negativo) o N+ (muy negativo), devolvemos 1
    elif x in ('N', 'N+'):
        return 1
    # NONE y NEU los filtraremos después, por eso no los incluimos aquí

In [ ]:
# Eliminamos los tweets neutros y sin polaridad definida
# ~df['Polarity'].isin([...]) significa "NO está en la lista"
# Nos quedamos solo con tweets claramente positivos o negativos

df = df[~df['Polarity'].isin(['NONE', 'NEU'])]

# Verificamos que solo quedan las polaridades que nos interesan: P, P+, N, N+
df['Polarity'].unique()

In [ ]:
# Aplicamos la función de transformación a toda la columna Polarity
# apply(): aplica una función a cada elemento de la columna
# Ahora Polarity contendrá solo 0 (positivo) o 1 (negativo)

df['Polarity'] = df['Polarity'].apply(polaridad_fun)

# Comprobamos que la transformación fue exitosa
df['Polarity'].unique()

#### Idioma
Nos quedamos con los tweets en español. Si no tuviésemos esa columna podríamos acudir a librerías como `langid` o `langdetect`.

In [ ]:
# Filtramos para quedarnos solo con tweets en español
# Aunque el corpus es español, puede haber tweets en otros idiomas

df = df[df['Lang'] == 'es']

In [ ]:
# Verificamos cuántos registros nos quedan después de los filtros aplicados
# Empezamos con 7219 tweets, veamos cuántos tenemos ahora
df.shape

#### Duplicados

In [ ]:
# Eliminamos tweets duplicados basándonos en la columna Content
# subset='Content': solo consideramos el contenido para detectar duplicados
# inplace=True: modificamos el DataFrame original sin crear una copia

df.drop_duplicates(subset = 'Content', inplace=True)

# Vemos cuántos tweets eliminamos (de 5066 a ...)
df.shape

#### Signos de puntuación
Eliminamos signos de puntuación: puntos, comas, interrogaciones, paréntesis

In [ ]:
# Visualizamos los primeros tweets antes de limpiar signos de puntuación
# Observa que tienen puntos, comas, interrogaciones, @menciones, etc.
df['Content'].head()

In [ ]:
# Importamos el módulo de expresiones regulares (regex)
import re

# Compilamos una expresión regular que captura todos los signos de puntuación y números
# Los paréntesis crean grupos de captura para cada símbolo
# \d+ captura uno o más dígitos
# Incluimos: . ; : ! ? ¿ @ , " ( ) [ ] y números

signos = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\¿)|(\@)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")

In [ ]:
# Importamos de nuevo para tener todo junto en esta celda
import re

# Compilamos la expresión regular con los signos de puntuación a eliminar
signos = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\¿)|(\@)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")

# Función que elimina signos de puntuación y convierte a minúsculas
def signs_tweets(tweet):
    # signos.sub('', tweet): sustituye todos los signos encontrados por cadena vacía
    # .lower(): convierte todo el texto a minúsculas
    return signos.sub('', tweet.lower())

# Aplicamos la función a toda la columna Content
df['Content'] = df['Content'].apply(signs_tweets)

# Visualizamos el resultado: tweets limpios, sin signos y en minúsculas
df['Content'].head()

#### Eliminamos links

In [ ]:
# Función para eliminar enlaces/URLs de los tweets
# Los enlaces no aportan información de sentimiento y pueden ser ruido

def remove_links(df):
    # Dividimos el tweet en palabras con split()
    # Si una palabra contiene 'http', la reemplazamos por el token '{link}'
    # join(): vuelve a unir las palabras en una cadena de texto
    return " ".join(['{link}' if ('http') in word else word for word in df.split()])

# Aplicamos la función a toda la columna
df['Content'] = df['Content'].apply(remove_links)

#### Otros
Podríamos hacer un preprocesado mucho más fino:
1. Hashtags
2. Menciones
3. Abreviaturas
4. Faltas de ortografía
5. Risas

## 3. Modelo
Para montar el modelo tendremos que seguir los siguientes pasos
1. Eliminamos las stopwords
2. Aplicamos un stemmer, SnowBall por ejemplo

#### Stopwords

In [ ]:
# Importamos la lista de stopwords (palabras vacías) en español desde NLTK
# Stopwords son palabras muy comunes que no aportan significado (el, la, de, en, y, etc.)
# Al eliminarlas, reducimos el ruido y mejoramos el rendimiento del modelo

from nltk.corpus import stopwords

# Cargamos el listado de stopwords en español
spanish_stopwords = stopwords.words('spanish')

In [ ]:
# Función para eliminar stopwords de un texto
def remove_stopwords(df):
    # Dividimos el texto en palabras con split()
    # Filtramos: solo conservamos palabras que NO están en spanish_stopwords
    # join(): reunimos las palabras filtradas en un texto
    return " ".join([word for word in df.split() if word not in spanish_stopwords])

In [ ]:
# Aplicamos la eliminación de stopwords a todos los tweets
df['Content'] = df['Content'].apply(remove_stopwords)

# Visualizamos cómo quedan los tweets sin palabras vacías
# Notarás que desaparecen palabras como "el", "la", "de", "en", etc.
df.head()

#### Stemmer

In [ ]:
# Importamos el stemmer de Snowball para español
# Stemming: proceso de reducir palabras a su raíz o lexema
# Ejemplo: "corriendo", "corrió", "correr" -> "corr"
# Esto ayuda al modelo a entender que son variantes de la misma palabra

from nltk.stem.snowball import SnowballStemmer

def spanish_stemmer(x):
    # Creamos una instancia del stemmer para español
    stemmer = SnowballStemmer('spanish')
    
    # Aplicamos el stemmer a cada palabra del texto
    # stem(word): reduce la palabra a su raíz
    return " ".join([stemmer.stem(word) for word in x.split()])

# Aplicamos stemming a todos los tweets
df['Content'] = df['Content'].apply(spanish_stemmer)

# Vemos el resultado: palabras reducidas a su raíz
# Ejemplo: "pensando" -> "pens", "gracias" -> "graci"
df['Content'].head()

#### Seleccionamos columnas
Nos quedamos con las columnas que nos interesan para el modelo

In [ ]:
# Nos quedamos solo con las columnas necesarias para el modelo
# Content: el texto del tweet (ya limpio y procesado)
# Polarity: la etiqueta (0=positivo, 1=negativo)

df = df[['Content', 'Polarity']]

In [ ]:
# Guardamos el dataset procesado en un archivo CSV
# Este archivo contiene los tweets limpios y listos para entrenar el modelo
# Es buena práctica guardar los datos procesados para poder reutilizarlos

df.to_csv('data/output/data_processed.csv')

#### Vectorizamos el dataset

In [ ]:
# Importamos CountVectorizer para convertir texto en vectores numéricos
# Los modelos de ML no entienden texto, necesitan números

from sklearn.feature_extraction.text import CountVectorizer

# Creamos el vectorizador con n-gramas de 1 y 2 palabras
# ngram_range=(1,2): captura palabras individuales (1-grama) y pares de palabras (2-gramas)
# Ejemplo: "muy bueno" se captura como ["muy", "bueno", "muy bueno"]
# Esto ayuda a captar contexto y expresiones compuestas

vectorizer = CountVectorizer(ngram_range=(1,2))

#### Montamos Pipeline
Modelos que suelen funcionar bien con pocas observaciones y muchas features son la Regresión logística el LinearSVC o Naive Bayes.

In [ ]:
# Importamos Regresión Logística, un algoritmo de clasificación efectivo para texto
from sklearn.linear_model import LogisticRegression

# Creamos un Pipeline que encadena transformaciones y el modelo
# Pipeline: ejecuta pasos secuenciales (vectorizar -> clasificar)
# Ventaja: simplifica el código y evita errores de data leakage

pipeline = Pipeline([
    ('vect', vectorizer),  # Paso 1: Convertir texto a vectores numéricos
    ('cls', LogisticRegression(max_iter=10000))  # Paso 2: Clasificar con Regresión Logística
    # max_iter=10000: número máximo de iteraciones para asegurar convergencia
])

# Definimos la grilla de hiperparámetros para probar diferentes combinaciones
parameters = {
    # max_features: número máximo de características (palabras/n-gramas) a considerar
    'vect__max_features': (5000, 10000),
    
    # penalty: tipo de regularización (L2 penaliza coeficientes grandes)
    "cls__penalty": ["l2"], 
    
    # C: inverso de la fuerza de regularización (menor C = más regularización)
    # Valores más altos = modelo más complejo, valores más bajos = modelo más simple
    "cls__C": [0.1, 0.5, 1.0, 5.0]
}

# GridSearchCV: prueba todas las combinaciones de hiperparámetros
# cv=5: validación cruzada con 5 folds (divide datos en 5 partes)
# n_jobs=-1: usa todos los núcleos del CPU para acelerar el proceso
# scoring='accuracy': métrica de evaluación (porcentaje de aciertos)

grid_search = GridSearchCV(pipeline,
                          parameters,
                          cv = 5,
                          n_jobs = -1,
                          scoring = 'accuracy')

#### Entrenamos

In [ ]:
# Entrenamos el modelo con todas las combinaciones de hiperparámetros
# X: df['Content'] - el texto de los tweets (features)
# y: df['Polarity'] - las etiquetas (0=positivo, 1=negativo)

# Este proceso puede tardar varios minutos porque:
# - Prueba 2 valores de max_features × 4 valores de C = 8 combinaciones
# - Cada combinación se evalúa con validación cruzada de 5 folds
# - Total: 8 × 5 = 40 entrenamientos

grid_search.fit(df['Content'], df['Polarity'])

In [ ]:
# Mostramos los resultados del entrenamiento

# best_params_: la mejor combinación de hiperparámetros encontrada
print("Best params:", grid_search.best_params_)

# best_score_: la mejor precisión (accuracy) obtenida con validación cruzada
# En este caso: ~76.9% de aciertos
print("Best acc:", grid_search.best_score_)

# best_estimator_: el pipeline completo con los mejores hiperparámetros
# Este es el modelo final que usaremos para hacer predicciones
print("Best model:", grid_search.best_estimator_)

In [ ]:
# Accedemos específicamente al clasificador (segundo paso del pipeline)
# Esto nos permite ver los parámetros del modelo de Regresión Logística
# 'cls' es el nombre que le dimos al clasificador en el pipeline

grid_search.best_estimator_['cls']

#### Guardamos el modelo

In [ ]:
# Guardamos el modelo entrenado en un archivo usando pickle
# pickle: serializa objetos de Python para poder guardarlos y cargarlos después

import pickle

# Abrimos un archivo en modo escritura binaria ('wb')
with open('data/output/finished_model.model', "wb") as archivo_salida:
    # dump(): guarda el mejor modelo (pipeline completo) en el archivo
    # Incluye tanto el vectorizador como el clasificador entrenado
    pickle.dump(grid_search.best_estimator_, archivo_salida)

# Ahora podemos cargar este modelo en cualquier momento sin tener que reentrenarlo

## 4. Predicciones
Realizar una predicción con un tweet que escojas

In [ ]:
# Importamos pickle para cargar el modelo guardado previamente
import pickle

In [ ]:
# Cargamos el modelo entrenado desde el archivo
# 'rb': modo lectura binaria (read binary)

with open('data/output/finished_model.model', "rb") as archivo_entrada:
    # load(): deserializa el objeto y lo carga en memoria
    # pipeline_importada contendrá el pipeline completo (vectorizador + clasificador)
    pipeline_importada = pickle.load(archivo_entrada)

In [ ]:
# Visualizamos el pipeline cargado para confirmar que es el correcto
# Debe mostrar CountVectorizer con max_features=10000 y LogisticRegression
pipeline_importada

In [ ]:
# Redefinimos todas las funciones de preprocesamiento
# Las necesitamos para limpiar nuevos tweets antes de hacer predicciones

# Función para eliminar signos de puntuación y convertir a minúsculas
def signs_tweets(tweet):
    return signos.sub('', tweet.lower())

# Función para eliminar URLs
def remove_links(df):
    return " ".join(['{link}' if ('http') in word else word for word in df.split()])

# Función para eliminar stopwords (palabras vacías)
def remove_stopwords(df):
    return " ".join([word for word in df.split() if word not in spanish_stopwords])

# Función para aplicar stemming (reducir palabras a su raíz)
from nltk.stem.snowball import SnowballStemmer

def spanish_stemmer(x):
    stemmer = SnowballStemmer('spanish')
    return " ".join([stemmer.stem(word) for word in x.split()])

#### Leemos el pipeline con el modelo

In [ ]:
# Creamos un tweet de ejemplo para probar el modelo
# Este tweet tiene sentimiento negativo (no estoy aprendiendo, malos profesores, etc.)
text = pd.Series('Bua, no estoy aprendiendo en el bootcamp. Qué malos profesores, me sobra tiempo')

# Convertimos a DataFrame
test_clean = pd.DataFrame(text, columns=['content'])

# Aplicamos todas las transformaciones en el orden correcto:

# Paso 1: Eliminar signos de puntuación
test_clean['content_clean'] = test_clean['content'].apply(signs_tweets)

# Paso 2: Eliminar enlaces (URLs)
test_clean['content_clean'] = test_clean['content_clean'].apply(remove_links)

# Paso 3: Eliminar stopwords
test_clean['content_clean'] = test_clean['content_clean'].apply(remove_stopwords)

# Paso 4: Aplicar stemming
test_clean['content_clean'] = test_clean['content_clean'].apply(spanish_stemmer)

In [ ]:
# Visualizamos el tweet original y el preprocesado
# Observa cómo el texto se ha simplificado pero mantiene las palabras clave
# "malos profesores" -> "mal profesor" (después del preprocesamiento)
test_clean

#### Predicciones de test

In [ ]:
# Extraemos el texto limpio que vamos a clasificar
# Este es el input que pasaremos al modelo
test_clean['content_clean']

In [ ]:
# Hacemos la predicción con el modelo cargado
# predict(): devuelve la clase predicha (0 o 1)
# 0 = Sentimiento positivo
# 1 = Sentimiento negativo

predictions = pipeline_importada.predict(test_clean['content_clean'])

# Añadimos la predicción como una nueva columna
test_clean['Polarity'] = pd.Series(predictions)

# Visualizamos el resultado: debería predecir 1 (negativo) para este tweet
test_clean

In [ ]:
# Obtenemos las probabilidades de cada clase
# predict_proba(): devuelve un array con [prob_clase_0, prob_clase_1]
# Esto nos da más información que solo la clase predicha

predictions = pipeline_importada.predict_proba(test_clean['content_clean'])

# Extraemos las probabilidades:
# predictions[0][0]: probabilidad de ser positivo (clase 0)
# predictions[0][1]: probabilidad de ser negativo (clase 1)

test_clean['Polarity_Pos'] = pd.Series(predictions[0][0])  # ~43% positivo
test_clean['Polarity_Neg'] = pd.Series(predictions[0][1])  # ~57% negativo

# Visualizamos el resultado completo
# El modelo está 57% seguro de que es negativo y 43% de que es positivo
# Como 0.57 > 0.43, clasifica como negativo (clase 1)
test_clean